In [1]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM

In [2]:
inputs = tensorflow.keras.Input(shape=(None,), dtype="int32")   # Input for variable length sequences of integers

x = Embedding(20000, 128)(inputs)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(64))(x)

In [4]:
# Add a classifier
outputs = Dense(1, activation="sigmoid")(x)
model = tensorflow.keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 128)      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,757,761 (10.52 MB)

 Trainable params: 2,757,761 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
(x_train, y_train), (x_val, y_val) = tensorflow.keras.datasets.imdb.load_data(
    num_words=20000
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
# Use pad_sequence to standardize sequence length:
# this will truncate sequences longer than 200 words and zero-pad sequences shorter than 200 words.
x_train = tensorflow.keras.utils.pad_sequences(x_train, maxlen=200)
x_val = tensorflow.keras.utils.pad_sequences(x_val, maxlen=200)

25000 Training sequences
25000 Validation sequences


In [6]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 194s 236ms/step - accuracy: 0.7442 - loss: 0.4983 - val_accuracy: 0.8301 - val_loss: 0.3755
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 204s 260ms/step - accuracy: 0.9014 - loss: 0.2624 - val_accuracy: 0.8672 - val_loss: 0.3171


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)

user_input = input("Enter your text: ")

tokenizer.fit_on_texts([user_input])
tokenized_input = tokenizer.texts_to_sequences([user_input])

padded_input = tensorflow.keras.utils.pad_sequences(tokenized_input, maxlen=200, padding='post', truncating='post', value=0.0)

predictions = model.predict(padded_input)

predicted_class = (predictions[0][0] > 0.5).astype("int32")

print(predictions)
if predicted_class == 1:
    print("Predicted sentiment: Positive")
else:
    print("Predicted sentiment: Negative")

Enter your text:  good


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[[0.6378386]]
Predicted sentiment: Positive
